In [ ]:
#%matplotlib widget
from spottunet.dataset.cc359 import Rescale3D, CC359, scale_mri
from dpipe.dataset.wrappers import apply, cache_methods
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import nibabel as nib
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)



data_path="/home/mlk/cc359"
voxel_spacing=[1, 0.95, 0.95]
df = pd.read_csv("meta.csv")

### Paper dataloading

In [ ]:
from dpipe.batch_iter import Infinite, load_by_random_id, unpack_args, multiply
from dpipe.im.shape_utils import prepend_dims
from spottunet.batch_iter import slicewise, SPATIAL_DIMS, get_random_slice, extract_patch, sample_center_uniformly

preprocessed_dataset = apply(Rescale3D(CC359(data_path), voxel_spacing), load_image=scale_mri)
dataset = apply(preprocessed_dataset, load_image=np.float32)

load_x = dataset.load_image
load_y = dataset.load_segm

ids_sampling_weights = None
slice_sampling_interval = 1
seed = 0xBadCafe
x_patch_size = y_patch_size = np.array([256, 256])
batch_size = 32
batches_per_epoch = 100

def get_random_patch_2d(image_slc, segm_slc, x_patch_size, y_patch_size):
    sp_dims_2d = (-2, -1)
    center = sample_center_uniformly(segm_slc.shape, y_patch_size, sp_dims_2d)
    x, y = extract_patch((image_slc, segm_slc, center), x_patch_size, y_patch_size, spatial_dims=sp_dims_2d)
    return x, y

x = load_by_random_id(dataset.load_image, dataset.load_segm, ids=df['id'],
                      weights=ids_sampling_weights, random_state=seed),
x,y = next(x[0])
print(x.shape,y.shape)

plt.imshow(x[75])
plt.show()
plt.imshow(y[75])
plt.show()


scaled_x = (x*255).astype(np.uint8)
scaled_x = scaled_x / 255.


diff = scaled_x[75]-x[75]
plt.imshow(diff)
plt.show()
print(diff.min(), diff.max())

In [ ]:
import numpy as np
arr = np.array([[534.57594992, 342.57558555],
 [530.33945514, 400.71141039],
 [542.48652573, 444.98639819],
 [547.99561513, 378.95523483],
 [423.66897131, 338.37503833],
 [423.57189916, 395.56296789],
 [418.43571045, 438.0447123 ],
 [418.32133933, 373.22089059]])

image_shape = [480,848]
minxy = np.min(arr,axis=0) #axis=1
print("minxy: "+str(minxy))
# box_corners_in_image: [N, 8, 2]
maxxy = np.max(arr,axis=0) #war box_corners_in_image), axis=1
print("maxxy: "+str(maxxy))
#bbox = np.concatenate([minxy, maxxy]) #axis = 1
bbox = np.stack((minxy, maxxy), axis=1)
print("bbox: "+str(bbox))

bbox[2:] = np.minimum(bbox[2:], image_shape[::-1])
bbox[:2] = np.maximum(bbox[:2], [0, 0])
print("bbox mit neuem min und max: "+str(bbox[j]))
anno["bbox"].append(bbox)
anno["alpha"].append(
    -np.arctan2(-final_box_preds[j, 1], final_box_preds[j, 0])
    + box3d_pts_2d[j, 6]
)
# anno["dimensions"].append(box3d_camera[j, [4, 5, 3]])
anno["dimensions"].append(box3d_pts_2d[j, 3:6])
anno["location"].append(box3d_pts_2d[j, :3])
anno["rotation_y"].append(box3d_pts_2d[j, 6])
anno["name"].append(class_names[int(label_preds[j])])
anno["truncated"].append(0.0)
anno["occluded"].append(0)
anno["score"].append(scores[j])

In [ ]:
for idx, inputs in enumerate(iterator()):
    print(inputs)

In [ ]:
for idx in df['id']:
    img_1 = preprocessed_dataset.load_spacing(idx)
    print(idx,img_1)

In [ ]:
## Viz Learning rate scheduler

from conf import CFG
import torch
from torch.optim.lr_scheduler import MultiplicativeLR
import matplotlib.pyplot as plt

optimizer = CFG.optim([torch.tensor(1)],lr=CFG.lr,weight_decay=CFG.wd)

scheduler = MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.96, )
lrs = []
for e in range(100):
    optimizer.step()
    lrs.append(scheduler.get_lr())
    scheduler.step()

plt.plot(lrs)
plt.show()

In [ ]:
img_1 = preprocessed_dataset.load_image("CC0030")
seg_1 = preprocessed_dataset.load_segm("CC0030")

out = np.zeros((150,269,269*2))
out[:,:,:269] = img_1
out[:,:,269:] = seg_1
create_animation(out)

In [ ]:
img_1 = preprocessed_dataset.load_image("CC0201")
    seg_1 = preprocessed_dataset.load_segm("CC0201")

    c,h,w = img_1.shape
    out = np.zeros((c,h,w*2))
    out[:,:,:269] = img_1
    out[:,:,269:] = seg_1
    create_animation(out)

In [ ]:
for idx in df['id']:
    img_1 = preprocessed_dataset.load_image(idx)
    seg_1 = preprocessed_dataset.load_segm(idx)

    c,h,w = img_1.shape
    out = np.zeros((c,h,w*2))
    out[:,:,:w] = img_1
    out[:,:,w:] = seg_1
    ani = create_animation(out)
    FFwriter = animation.FFMpegWriter(fps=10)
    ani.save(f'dataset_viz/{idx}.mp4', writer = FFwriter)

In [ ]:
df[df['id']=="CC0326"]['tomograph_model'].values[0]

In [ ]:
import cv2
for idx in df['id']:
    print(idx)
    img_1 = preprocessed_dataset.load_image(idx)
    seg_1 = preprocessed_dataset.load_segm(idx)
    img_1 = np.transpose(img_1,(2,0,1))
    seg_1 = np.transpose(seg_1,(2,0,1))
    model = df[df['id']==idx]['tomograph_model'].values[0]
    tsla  = df[df['id']==idx]['tesla_value'].values[0]
    c,h,w = img_1.shape
    out = np.zeros((1,h,w*2))
    out[:,:,:w] = img_1[c//2]
    out[:,:,w:] = seg_1[c//2]
    #plt.figure(figsize=(20,10))
    #plt.imshow(out.squeeze())
    fn_out = f'dataset_plots3/{idx}_{model}-{str(tsla)}.jpg'
    print(fn_out,  out.shape)
    cv2.imwrite(fn_out,(out*255).astype(np.uint8).squeeze())

### Custom dataloading

In [ ]:
from spottunet.dataset.cc359 import *
from spottunet.split import one2all
from spottunet.torch.module.unet import UNet2D
from spottunet.utils import sdice
from dpipe.im.metrics import dice_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler 

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from monai import transforms as T
from monai.transforms import Compose, apply_transform
from fastprogress.fastprogress import master_bar, progress_bar

import json
import nibabel as nib
import pandas as pd
import numpy as np
from scipy import ndimage
from dpipe.im.shape_ops import zoom
import cv2
import os
import gc
from collections import defaultdict
from PIL import Image
import seaborn as sns

In [ ]:
from dataset.augment import augment_transform

hparams = dict(
            translate_const=int(256 * 0.020),
            img_mean=tuple([0]),
            )

aug = augment_transform(["TranslateXRel"],"rand-m9-mstd0.5",hparams)
aug(Image.fromarray(np.zeros((3,256,256),dtype=np.uint8), mode="RGB"),Image.fromarray(np.zeros((3,256,256),dtype=np.uint8), mode="RGB"))

In [ ]:
import torch

sgd = torch.load("/media/mlk/New Volume/Lab/domain_shift_anatomy/dart_results/baseline_default/experiment_3/model.pth", map_location="cpu")
adam = torch.load("baseline_results/baseline_focal_lovasz_adam_rand_aug_default_v1/mode_3/e_39.pth", map_location="cpu")['model_state_dict']

In [ ]:
for k in sgd.keys():
    fig, axes = plt.subplots(1,2, figsize=(10,5))
    plt.title(k)
    sns.histplot(np.array(sgd[k]).reshape(-1), ax=axes[0], )
    sns.histplot(np.array(adam[k]).reshape(-1), ax=axes[1], )
    plt.show()

In [ ]:
sns.histplot(np.array(sgd['init_path.1.conv_path.0.bn.weight']).reshape(-1))
plt.show()
sns.histplot(np.array(adam['init_path.1.conv_path.0.bn.weight']).reshape(-1))
plt.show()

In [ ]:
f, ax = plt.subplots()
sns.violinplot(data=[np.array(sgd['init_path.1.conv_path.0.bn.weight']).reshape(-1),np.array(adam['init_path.1.conv_path.0.bn.weight']).reshape(-1)])
sns.despine(offset=10, trim=True);

In [ ]:
sgd.keys()

In [ ]:
def _grid_distortion_level_to_arg(level, _hparams):
    level = (level / (10.+1)) * 0.5
    return level,

_grid_distortion_level_to_arg(random.gauss(9, 0.5),None)

In [ ]:
from intensity_normalization.normalize.fcm import FCMNormalize
import glob

for fn, fn_brain_mask in zip(sorted(glob.glob(f"{CFG.dataset_path}/images_scaled_voxel_spacing/*")),
                             sorted(glob.glob(f"{CFG.dataset_path}/robex_masks_svsp/*"))):

    #fn = f"{CFG.dataset_path}/images_scaled_voxel_spacing/CC0001_philips_15_55_M.nii.gz"
    #fn_brain_mask = f"{CFG.dataset_path}/robex_masks_svsp/CC0001_philips_15_55_M.nii.gz"
    fcm_norm = FCMNormalize(tissue_type="csf")
    file = nib.load(fn)
    file_mask = nib.load(fn_brain_mask)

    normed_file = fcm_norm(file,file_mask)
    
    outfile = fn.replace("images_scaled_voxel_spacing", "fcm_norm_csf")
    nib.save(normed_file, outfile)
    
    tissue_membership = nib.Nifti1Image(
                fcm_norm.tissue_membership,
                normed_file.affine,
                normed_file.header,
            )
    outfile_tissue_membership = outfile[:-7] + 'csf_membership' + outfile[-7:]
    tissue_membership.to_filename(outfile_tissue_membership)

In [ ]:
plt.imshow(fcm_norm.tissue_membership[130])
plt.show()

In [ ]:
%reload_ext autoreload
%autoreload 2
from configs.config import CFG
from dataset.dataloader import *
from dataset.dataloader_utils import *
from dataset.augment import get_transforms, get_test_transforms
import seaborn as sns

from PIL import Image, ImageEnhance

cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
train_dataset = CC359_Dataset(CFG,df=cc359_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,transforms=None,#get_transforms("default"),
                                  mode="test", cache=False,)
cc359_df.head()

In [ ]:
from pathlib import Path
from dpipe.io import load
path_base = Path('baseline_results/baseline_lovasz_default')

meta = pd.read_csv(f"meta.csv",delimiter=",", index_col='id')
meta.head()

all_sdices = []
for s in sorted(cc359_df['fold'].unique()):
    sdices = load(path_base / f'mode_{s}/sdice_score.json')
    all_sdices.append(sdices)

def id_to_scanner(id):
    df = cc359_df[cc359_df['id']==id]
    return df['tomograph_model'].values[0] + str(df['tesla_value'].values[0])
test = "CC0231"
id_to_scanner(test)


In [ ]:
from dataset.rand_augment import *

_RAND_TRANSFORMS = [
    'AutoContrast',
    'Rotate',
    'Posterize',
    'Solarize',
    'SolarizeAdd',
    'Contrast',
    'Brightness',
    'Sharpness',
    'ShearX',
    #'ShearY',
    'TranslateXRel',
    #'TranslateYRel',
    #'Gamma',
    #'GridDistortion'
    #'Cutout'  # NOTE I've implement this as random erasing separately
]

In [ ]:
_RAND_TRANSFORMS

In [ ]:


train_dataset = CC359_Dataset(CFG,df=cc359_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,transforms=None,#get_transforms("default"),
                                  mode="test", cache=False,)

for x, y, id in [train_dataset[0]]:
    sdices = [np.round(sdices[id],3) for sdices in all_sdices if id in sdices.keys()] 
    x_slice = Image.fromarray(np.array(x.squeeze()[130])*255.).convert('L')
    y_slice = Image.fromarray(np.array(y.squeeze()[130])*255.).convert('L')
    """fig, axes = plt.subplots(1,3, figsize=(30,10))
    axes[0].set_title(f"{x.shape}, {id} {id_to_scanner(id)}: {sdices}")
    inp = axes[0].imshow(x_slice)
    plt.colorbar(inp,ax=axes[0])
    
    
    aug_slice = ImageEnhance.Brightness(x_slice).enhance(1.7)
    inp2 = axes[1].imshow(aug_slice)
    plt.colorbar(inp2,ax=axes[1])
    
    inten = A.Posterize(p=1.,num_bits=3)
    aug_slice = inten(image=np.array(x_slice))['image']
    inp3 = axes[2].imshow(aug_slice)
    plt.colorbar(inp3,ax=axes[2])
    plt.show()"""
    
    fig, axs = plt.subplots(2,5, figsize=(15, 6))
    fig.subplots_adjust(hspace =0.15, wspace=.001)
    axs = axs.ravel()
    aa_params = dict(
            translate_const=int(256 * 0.020),
            img_mean=tuple([0]),
            #interpolation=str_to_pil_interp(interpolation)
            )
    for i, name in enumerate(_RAND_TRANSFORMS):
        op = NAME_TO_OP[name]
        args = LEVEL_TO_ARG[name] 
        factor = args(9,aa_params) if args is not None else tuple()
        aug_slice, y_slice = op(x_slice,y_slice, *factor)
        axs[i].set_title(name)
        axs[i].imshow(aug_slice, "gray")
        axs[i].axis('off')
    plt.savefig('aug_viz.png', dpi=300)    
    plt.show()
    break

In [ ]:
from skimage.restoration import denoise_tv_chambolle
tv_denoised = denoise_tv_chambolle(np.array(x_slice), weight=0.05)
plt.figure(figsize=(30,10))
plt.imshow(tv_denoised)
plt.show()

In [ ]:
from intensity_normalization.normalize.fcm import FCMNormalize

fcm_norm = FCMNormalize(tissue_type="wm")

from monai import transforms as T

monai_T = T.Compose([T.CenterSpatialCropd(keys=("image"),roi_size=[256,256,256]),
                            T.SpatialPadd(keys=("image"),spatial_size=[256,256,256])])

x = nib.load("/home/mlk/cc359/images_scaled_voxel_spacing/CC0017_philips_15_50_F.nii.gz")

tfmed = monai_T({'image':x.get_fdata()[None,:,:,:]})       
x_tfms = tfmed['image'].squeeze()
normalized = fcm_norm(x)

In [ ]:
x = nib.load("/home/mlk/cc359/images_scaled_voxel_spacing/CC0001_philips_15_55_M.nii.gz")
x2 = nib.load("/home/mlk/cc359/images_scaled_voxel_spacing/CC0261_ge_15_59_F.nii.gz")

fig, axes = plt.subplots(1,4, figsize=(40,10))
axes[0].imshow(x.get_fdata().transpose(2,0,1)[130])

sns.distplot(x.get_fdata().reshape(-1), ax=axes[1])
#axes[1].set_xlim(0,1)

axes[2].imshow(x2.get_fdata().transpose(2,0,1)[130])

sns.distplot(x2.get_fdata().reshape(-1), ax=axes[3])
#axes[3].set_xlim(0,1)
plt.show()

In [ ]:
sns.set_style("darkgrid")
x_1 = scale_mri(x.get_fdata().reshape(-1),q_min=0,q_max=100)
#x_1 = x_1[x_1>0.1]
x_2 = scale_mri(x2.get_fdata().reshape(-1),q_min=0,q_max=100)
#x_2 = x_2[x_2>0.1]
sns.kdeplot(data=[x_1,x_2], fill=True, palette="crest",
   alpha=.5, linewidth=0,)
plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(30,10))
axes[0].set_title(f"{x.shape}, {id} {id_to_scanner(id)}: {sdices}")
axes[0].imshow(x_tfms.transpose(2,0,1)[130])

sns.distplot(x.get_fdata().reshape(-1), ax=axes[1])
#axes[1].set_xlim(0,1)

sns.distplot(normalized.get_fdata().reshape(-1), ax=axes[2])
axes[2].set_xlim(0,1)
plt.show()

In [ ]:
for x, y, id in train_dataset:
    sdices = [np.round(sdices[id],3) for sdices in all_sdices if id in sdices.keys()] 
    
    fig, axes = plt.subplots(1,3, figsize=(30,10))
    axes[0].set_title(f"{x.shape}, {id} {id_to_scanner(id)}: {sdices}")
    axes[0].imshow(x.squeeze()[130])
    
    sns.distplot(x[y==1.], ax=axes[1])
    axes[1].set_xlim(0,1)
    
    sns.distplot(normalized[y==1.], ax=axes[2])
    axes[2].set_xlim(0,1)
    plt.show()
    break

### Calc dataset mean and std

In [ ]:
mean = 0.
std = 0.
nb_samples = 0.
for data, y, id in train_dataset:
    data = data.unsqueeze(0)
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

In [ ]:
mean /= nb_samples
std /= nb_samples

In [ ]:
print(mean,std)

In [ ]:
import torch

state_dict = torch.load("baseline_results/baseline_focal_lovasz_adam_default/mode_4/e_39.pth",map_location="cpu")['model_state_dict']

In [ ]:
conv_weights = []
for k in state_dict.keys():
    if "conv" in k and "layer" in k and "weight" in k:
        conv_weights.append(np.array(state_dict[k]).reshape(-1))
sns.displot(conv_weights)
plt.show()

In [ ]:
from skimage.restoration import denoise_tv_chambolle

for x,y in test_dataset:
    b,c,h,w = x.shape
    
    x_aug = denoise_tv_chambolle(np.array(x), weight=0.1)
    
    plt.figure(figsize=(20,10))
    plt.imshow(x.squeeze(), "gray")
    plt.show()
    plt.figure(figsize=(20,10))
    plt.imshow(x_aug.squeeze(), "gray")
    plt.show()
    break

In [ ]:
x = load_by_random_id(dataset.load_image, dataset.load_segm, ids=df['id'],
                      weights=ids_sampling_weights, random_state=seed),
x,y = next(x[0])
x = x[75]
y = y[75]

In [ ]:
x_aug = denoise_tv_chambolle(np.array(x), weight=0.1)
plt.figure(figsize=(20,10))
plt.imshow(x, "gray")
plt.show()
plt.figure(figsize=(20,10))
plt.imshow(x_aug, "gray")
plt.show()

In [ ]:
np.sum(y==1)/np.sum(y==0)

In [ ]:
plt.title("Original")
plt.imshow(x, "gray")
plt.show()

In [ ]:
from PIL import Image
x = Image.fromarray(np.array(x)*255).convert('L')
y = Image.fromarray(np.array(y)*255).convert('L')

In [ ]:
from rand_augment import *
_LEVEL_DENOM = 10.
_RAND_TRANSFORMS = [
    'AutoContrast',
    'Equalize',
    'Invert',
    'Rotate',
    'Posterize',
    'Solarize',
    'SolarizeAdd',
    'Color',
    'Contrast',
    'Brightness',
    'Sharpness',
    'ShearX',
    'ShearY',
    'TranslateXRel',
    'TranslateYRel',
    #'Cutout'  # NOTE I've implement this as random erasing separately
]

_FILL = (128, 128, 128)

_HPARAMS_DEFAULT = dict(
    translate_const=250,
    img_mean=_FILL,
)
mean = [0.5]

hparams = dict(
            translate_const=int(256 * 0.45),
            img_mean=tuple([min(255, round(255 * x)) for x in mean]),
            )

config_str = "rand-m9-mstd0.5"

magnitude = _LEVEL_DENOM  # default to _LEVEL_DENOM for magnitude (currently 10)
num_layers = 2  # default to 2 ops per image
weight_idx = None  # default to no probability weights for op choice
transforms = _RAND_TRANSFORMS
config = config_str.split('-')
assert config[0] == 'rand'
config = config[1:]
for c in config:
    cs = re.split(r'(\d.*)', c)
    if len(cs) < 2:
        continue
    key, val = cs[:2]
    if key == 'mstd':
        # noise param / randomization of magnitude values
        mstd = float(val)
        if mstd > 100:
            # use uniform sampling in 0 to magnitude if mstd is > 100
            mstd = float('inf')
        hparams.setdefault('magnitude_std', mstd)
    elif key == 'mmax':
        # clip magnitude between [0, mmax] instead of default [0, _LEVEL_DENOM]
        hparams.setdefault('magnitude_max', int(val))
    elif key == 'inc':
        if bool(val):
            transforms = _RAND_INCREASING_TRANSFORMS
    elif key == 'm':
        magnitude = int(val)
    elif key == 'n':
        num_layers = int(val)
    elif key == 'w':
        weight_idx = int(val)
    else:
        assert False, 'Unknown RandAugment config section'
ra_ops = rand_augment_ops(magnitude=magnitude, hparams=hparams, transforms=transforms)
choice_weights = None if weight_idx is None else _select_rand_weights(weight_idx)

In [ ]:
NAME_TO_OP.keys()

In [ ]:
ra_ops

In [ ]:
from rand_augment import *


for name,augment_fkt in zip(NAME_TO_OP.keys(),ra_ops):
    
    x_aug,y_aug = augment_fkt(x, y)
    plt.title(augment_fkt.name)
    plt.imshow(x_aug, "gray")
    plt.show()

In [ ]:
from rrc import *

scale = tuple((0.08, 1.0))  # default imagenet scale range
ratio = tuple((3./4., 4./3.)) # default imagenet ratio range
mean = [0.5]
_pil_interpolation_to_str = {
    Image.NEAREST: 'nearest',
    Image.BILINEAR: 'bilinear',
    Image.BICUBIC: 'bicubic',
    Image.BOX: 'box',
    Image.HAMMING: 'hamming',
    Image.LANCZOS: 'lanczos',
}
_str_to_pil_interpolation = {b: a for a, b in _pil_interpolation_to_str.items()}


def str_to_pil_interp(mode_str):
    return _str_to_pil_interpolation[mode_str]

auto_augment = "rand-m9-mstd0.5"
interpolation = 'random'
rrc = RandomResizedCropAndInterpolation((256,256), scale=scale, ratio=ratio, interpolation=interpolation)

plt.title("RandomResizeCrop")
plt.imshow(rrc(x,y)[0], "gray")
plt.show()

In [ ]:
for x,y,idx in test_dataset:
    print(idx,x.shape,x.min(),x.max())
    c,h,w = x.shape
    out = np.zeros((1,h,w*2))
    out[:,:,:w] = x[c//2]
    out[:,:,w:] = y[c//2]
    plt.figure(figsize=(20,10))
    plt.imshow(out.squeeze())
    plt.show()
    
    x1 = preprocessed_dataset.load_image(idx)
    y1 = preprocessed_dataset.load_segm(idx)

    print(idx,x1.shape,x1.min(),x1.max())
    c,h,w = x.shape
    out = np.zeros((1,h,w*2))
    out[:,:,:w] = x1[c//2]
    # out[:,:,w:] = y1[c//2]
    plt.figure(figsize=(20,10))
    plt.imshow(out.squeeze())
    plt.show()
    
    x_diff = x-x1
    print(x_diff.min(),x_diff.max())

In [ ]:
from dpipe.io import load

In [ ]:
from intensity_normalization.normalize.fcm import FCMNormalize

fcm_norm = FCMNormalize(tissue_type="wm")



voxel_spacing=CFG.voxel_spacing
def scale_voxel_spacing(idx, img, segm, sample_voxel_spacing):
    sample_vxsp = sample_voxel_spacing
    scale_factor = np.array(sample_vxsp) / np.array(voxel_spacing)
    scale_factor = np.nan_to_num(scale_factor, nan=1)
    img = zoom(img, scale_factor, order=3)
    segm = zoom(segm, scale_factor, order=3)
    return img,segm

img_fns, mask_fns = [],[]
for i,idx in enumerate(cc359_df['id']):
    img_fn = cc359_df['MRI'][i]
    seg_fn = cc359_df['brain_mask'][i]
    sample_voxel_spacing = [cc359_df['x'][i], cc359_df['y'][i], cc359_df['z'][i]]
    
    img_file = nib.load(f"{CFG.dataset_path}/{img_fn}")
    image = img_file.get_fdata()
    seg_file = nib.load(f"{CFG.dataset_path}/{seg_fn}")
    seg = seg_file.get_fdata() 
    
    image, seg = scale_voxel_spacing(idx,image,seg,sample_voxel_spacing)
    img_file.data = image
    #seg_file.data = seg
    normalized = fcm_norm(img_file)
    print(img_fn,image.min())
    img_out = f"{CFG.dataset_path}/{img_fn.replace('images','images_svs_norm')}"
    #msk_out = f"{CFG.dataset_path}/{seg_fn.replace('masks','masks_svs_norm')}"
    nib.save(normalized,img_out)
    #nib.save(seg_file,msk_out)
    img_fns.append(img_fn.replace('images','images_svs_norm'))
    #mask_fns.append(seg_fn.replace('masks','masks_svs_norm'))

In [ ]:
cc359_df['MRI_scaled_voxel_spacing'] = img_fns
cc359_df['brain_mask_scaled_voxel_spacing'] = mask_fns

In [ ]:
cc359_df

In [ ]:
cc359_df.to_csv(f"{CFG.dataset_path}/meta.csv")

In [ ]:
from spottunet.dataset.cc359 import *
from spottunet.split import one2all
from spottunet.torch.module.unet import UNet2D
from spottunet.utils import sdice
from dpipe.im.metrics import dice_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler 

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from PIL import Image

from monai import transforms as T
from monai.transforms import Compose, apply_transform
from fastprogress.fastprogress import master_bar, progress_bar

import json
import nibabel as nib
import pandas as pd
import numpy as np
from scipy import ndimage
from dpipe.im.shape_ops import zoom
import cv2
import os
import gc
from collections import defaultdict
from pathlib import Path
import segmentation_models_pytorch as smp

import matplotlib.pyplot as plt

from dataset.dataloader import *
from dataset.loader import *
# 
cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
seed = 0xBadCafe
val_size = 4
n_experiments = len(cc359_df.fold.unique())
split = one2all(df=cc359_df,val_size=val_size)[:n_experiments]
train_df = cc359_df.iloc[split[0][0]].reset_index()

"""train_dataset = CC359_Dataset(df=train_df,root_dir=CFG.dataset_path,
                              voxel_spacing=CFG.voxel_spacing,transforms=tfms,
                              mode="train", cache=True)"""

valid_df = cc359_df.iloc[split[0][1]].reset_index()

valid = CC359_Dataset(df=valid_df,root_dir=CFG.dataset_path,
                              voxel_spacing=CFG.voxel_spacing,transforms=None,
                              mode="val", cache=False)

In [ ]:
x,y,_id = next(iter(valid))
plt.imshow(x[180].squeeze(), "gray")
plt.show()
plt.imshow(y[180])
plt.show()

### Visualize Stats

In [ ]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

df = pd.read_csv(f"meta++.csv",delimiter=",")
data_path="/media/mlk/New Volume/conp-dataset/projects/calgary-campinas/CC359"

df["max_intensity"] = np.zeros(len(df))
df["size_s"] = np.zeros(len(df))
df["size_l"] = np.zeros(len(df))

for fn in os.listdir(os.path.join(data_path,"images")):
    df_fn = os.path.join("images",fn)
    fn_p = os.path.join(data_path,"images",fn)
    raw_img_data = nib.load(fn_p).get_fdata()
    np.unique(raw_img_data)
    df.loc[df["MRI"]==df_fn,"max_intensity"] = raw_img_data.max()
    min_s, max_s = np.unique(raw_img_data.shape)
    df.loc[df["MRI"]==df_fn,"size_s"] = min_s
    df.loc[df["MRI"]==df_fn,"size_l"] = max_s    

In [ ]:
for td in tds:
    df[td] = np.zeros(len(df))

In [ ]:
from pathlib import Path
from dpipe.io import load
path_base = Path('baseline_results/baseline_lovasz_default')

meta = pd.read_csv(f"meta.csv",delimiter=",", index_col='id')
meta.head()

all_sdices = {}
for s in sorted(meta['fold'].unique()):
    sdices = load(path_base / f'mode_{s}/sdice_score.json')
    all_sdices[s] = sdices

tds = ["siemens15","siemens3","ge15","ge3","philip15","philips3"]
for fold in all_sdices.keys():
    sdices = all_sdices[fold]
    for id in sdices.keys():
        dice = sdices[id]
        df.loc[df["id"]==id, tds[fold]] = np.round(dice,4)
        

In [ ]:
df

In [ ]:
from sklearn.linear_model import LinearRegression

coefs = []
for td in tds:
    df_subset = df.drop(index=df[df[td]==0.0].index)
    X = df_subset.drop(columns=["id","tomograph_model","tesla_value"])
    X = X.drop(columns=tds)
    y = df_subset[td]
    reg = LinearRegression().fit(X, y)
    print(reg.score(X,y))
    print(reg.coef_, reg.intercept_)
    coefs.append(reg.coef_)

In [ ]:
pd.DataFrame(data=coefs, index=tds, columns=X.columns)

In [ ]:
g = sns.boxplot(data=df, x="fold", y="size_0")
plt.show()
g = sns.boxplot(data=df, x="fold", y="size_1")
plt.show()
g = sns.boxplot(data=df, x="fold", y="size_2")
plt.show()

In [ ]:
import seaborn as sns

g = sns.boxplot(data=df, y="max_intensity", x="fold", )


In [ ]:


for fn in df.loc[df['fold']==5,"MRI"]:
    print(fn,raw_img_data.max())
    df_fn = os.path.join(fn)
    fn_p = os.path.join(data_path,fn)
    raw_img_data = nib.load(fn_p).get_fdata()
    plt.imshow(raw_img_data[75])
    plt.show()
    sns.displot(raw_img_data.reshape(-1)[raw_img_data.reshape(-1)!=0.0], log_scale=(False,True))
    plt.show()
    

In [ ]:
CC0027_philips_15_41_M.nii.gz
CC0255_ge_15_57_F.nii.gz

In [ ]:
x1 = nib.load(os.path.join(data_path,f"images/CC0027_philips_15_41_M.nii.gz")).get_fdata().transpose(2,0,1)

x2 = nib.load(os.path.join(data_path,f"images/CC0255_ge_15_57_F.nii.gz")).get_fdata().transpose(2,0,1)

print(x1.shape,x1.dtype,x1.min(),x1.max(),)
print(x2.shape,x2.dtype,x2.min(),x2.max())
plt.imshow(x1[130])
plt.show()
plt.imshow(x2[130])
plt.show()

In [ ]:
def scale_mri(image, q_min=1, q_max=99):
        #image = np.clip(np.float32(image), *np.percentile(np.float32(image), [q_min, q_max]))
        image -= np.min(image)
        image /= np.max(image)
        return np.float32(image)

from monai import transforms as T
monai_T = T.Compose([T.CenterSpatialCropd(keys=("image","seg"),roi_size=[256,256,256]),
                         T.SpatialPadd(keys=("image","seg"),spatial_size=[256,256,256])])

fn = "CC0255_ge_15_57_F.nii.gz"
fn2 = "CC0255_ge_15_57_F_ss.nii.gz"
raw_img_data = nib.load(os.path.join(data_path,f"images/{fn}")).get_fdata().transpose(2,0,1)
raw_seg_data = nib.load(os.path.join(data_path,f"masks/{fn2}")).get_fdata().transpose(2,0,1)
raw_img_data = scale_mri(raw_img_data)
tfmed = monai_T({'image':raw_img_data[None,:,:,:],
                 'seg':raw_seg_data[None,:,:,:]})       
raw_img_data = tfmed['image'].squeeze()
raw_seg_data = tfmed['seg'].squeeze()

#x1 = scale_mri(x1)
tfmed = monai_T({'image':x1[None,:,:,:],
                 'seg':x1[None,:,:,:]})       
x1 = tfmed['image'].squeeze()

preds = np.load(os.path.join("predictions/baseline23_2class_lovasz_default/mode_3",f"{fn.split('_')[0]}.npy"))

combined = np.concatenate((raw_img_data,raw_seg_data*1,preds*1), axis=2)
print(raw_img_data.max())
create_animation(np.concatenate((raw_img_data,x1), axis=2))

In [ ]:

print(raw_img_data.shape,raw_img_data.min(),raw_img_data.max())
fig, (ax,bx,cx) = plt.subplots(1,3, figsize=(10,20))
ax.imshow(raw_img_data[2], "gray")
bx.imshow(raw_seg_data[2], "gray")
cx.imshow(preds[2], "gray")
plt.show()

fig, (ax,bx,cx) = plt.subplots(1,3, figsize=(10,20))
ax.imshow(raw_img_data[70], "gray")
bx.imshow(raw_seg_data[70], "gray")
cx.imshow(preds[70], "gray")
plt.show()
fig, (ax,bx,cx) = plt.subplots(1,3, figsize=(10,20))
ax.imshow(raw_img_data[130], "gray")
bx.imshow(raw_seg_data[130], "gray")
cx.imshow(preds[130]*255, "gray")
plt.show()
print(raw_img_data[199].max())
fig, (ax,bx,cx) = plt.subplots(1,3, figsize=(10,20))
ax.imshow(raw_img_data[199], "gray")
bx.imshow(raw_seg_data[199], "gray")
cx.imshow(preds[199], "gray")
plt.show()

In [ ]:
import albumentations as A


In [ ]:
import albumentations as A
A.Posterize(num_bits=(0,4))
fig, (ax,bx,cx) = plt.subplots(1,3, figsize=(10,20))
ax.imshow(tmf(image=raw_img_data[199])['image'], "gray")
bx.imshow(tmf(image=raw_seg_data[199])['image'], "gray")
cx.imshow(tfm2(image=raw_img_data[199])['image'], "gray")
plt.show()

In [ ]:
sns.displot(x1.reshape(-1)[x1.reshape(-1)!=0.0], log_scale=(False,True))
plt.show()
sns.displot(x2.reshape(-1)[x2.reshape(-1)!=0.0], log_scale=(False,True))
plt.show()

In [ ]:

sns.displot(scale_mri(x1).reshape(-1)[x1.reshape(-1)!=0], log_scale=(False,True))
plt.show()
sns.displot(scale_mri(x2).reshape(-1)[x2.reshape(-1)!=0], log_scale=(False,True))
plt.show()

In [ ]:
noise = np.zeros((256,256))
noise[:,30:50] = np.arange(0,20)[::-1]
plt.imshow(noise)
plt.show()

In [ ]:
!pip install plotly

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()